In [1]:
import cv2
import numpy as np
import time
import os
import math

import track_hands as TH

save_delay = 0
motion_count = []
fist = 0
max_min_count = 0
save_count = 0
h_px = 720
w_px = 1280
h_px_overlay_upto = 130
w_px_overlay_upto = 1280
y1_read = 140
max_min_l = 5
max_min_r = 115
cls_l = 125
cls_r = 219
sav_l = 230
sav_r = 308
thick_dec_l = 326
thick_dec_r = 368
thick_inc_l = 432
thick_inc_r = 474
white_l = 508
white_r = 589
yellow_l = 603
yellow_r = 680
red_l = 694
red_r = 770
green_l = 784
green_r = 859
blue_l = 875
blue_r = 950
black_l = 967
black_r = 1039
erase_l = 1065
erase_r = 1165
exit_l = 1200
exit_r = 1280

text_write_w = 1055
text_select_w = 1020
text_client_w = 5
text_size_w = 935
text_h = 712
text_font_w = 387
text_font_h = 91
text_paint_save_h = 430
text_paint_save_w = 255
size_scale = 0.5

brush_thickness = 10
eraser_thickness = 20
thick_str = "10"

channel = 3
currentT=0
previousT =0

image_canvas =np.zeros((h_px,w_px,channel), np.uint8)
header_img = "Images720"
header_img_list = os.listdir(header_img)
overlay_image =[]
for i in header_img_list:
    image = cv2.imread(f'{header_img}/{i}')
    overlay_image.append(image)
default_overlay= overlay_image[0]

def rescale_frame(frame, percent):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

print(header_img_list)

cap = cv2.VideoCapture(1)
cap.set(3,w_px)
cap.set(4,h_px)
cap.set(cv2.CAP_PROP_FPS, 90)
draw_color = (0,0,0)
detector= TH.handDetector()

xp = 0
yp = 0

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)

    if(h_px == 1080):
        cv2.namedWindow('paint', cv2.WND_PROP_FULLSCREEN)
        # cv2.setWindowProperty('paint', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        frame = rescale_frame(frame, 150)
    frame[0:h_px_overlay_upto,0:w_px_overlay_upto] = default_overlay
    frame = detector.findHands(frame, draw=True)
    landmark_list = detector.findPosition(frame, draw =False)
    cv2.putText(frame, thick_str, (text_font_w,text_font_h), fontFace= cv2.FONT_HERSHEY_COMPLEX, color= (0,0,0), thickness=2, fontScale=size_scale)
    if(len(landmark_list)!=0):
        x0, y0 = landmark_list[4][1:] #thumb
        x1, y1 = landmark_list[8][1:] #index
        x2, y2 = landmark_list[12][1:] #middle
        x3, y3 = landmark_list[16][1:] #ring

        my_fingers = detector.fingerStatus()
        
        if(my_fingers[0] and my_fingers[1] and my_fingers[2] and my_fingers[3] and my_fingers[4]): #clear screen
            xp = 0
            yp = 0
            save_delay = 0
            motion_detected = True
            fist = 0
            motion_dir = detector.detect_motion()
            if (motion_dir != None):
                motion_count.append(motion_dir)
            if (len(motion_count)>=5):
                for i in range (1, len(motion_count)):
                    if(motion_count[i] != motion_count[i-1]):
                        motion_count = []
                        motion_detected = False
                        break
                if (motion_detected and motion_count[0] == "Left"):
                    image_canvas =np.zeros((h_px,w_px,channel), np.uint8)
                    default_overlay = overlay_image[10]
                        
        if (my_fingers[0] and not my_fingers[1] and not my_fingers[2] and not my_fingers[3] and not my_fingers[4]): #Save
            xp = 0
            yp = 0
            cv2.putText(frame, "Painting Saved", (text_paint_save_w,text_paint_save_h), fontFace= cv2.FONT_HERSHEY_COMPLEX, color= (0,0,0), thickness=4, fontScale=3)
            if (save_delay == 0):
                save_delay = 30
                motion_count = []
                fist = 0
                save_dir = 'S:/Chandigarh University/7th Semester/Python Codes/Air Painting/Saves/save'+str(save_count)+'.png'
                gray = cv2.cvtColor(image_canvas, cv2.COLOR_BGR2GRAY)
                mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
                result = np.dstack((image_canvas, mask))
                if cv2.imwrite(save_dir, result):
                    default_overlay = overlay_image[11]
                save_count += 1
            else:
                save_delay -= 1
            
        if (my_fingers[0] and my_fingers[1] and not my_fingers[2] and not my_fingers[3] and not my_fingers[4]): #Thickness
            xp = 0
            yp = 0
            save_delay = 0
            fist = 0
            motion_count = []
            cv2.putText(frame, 'Size Selection Mode', (text_size_w,text_h), fontFace=cv2.FONT_HERSHEY_COMPLEX, color= (0,255,255), thickness=2, fontScale=1)
            cv2.line(frame, (x0,y0), (x1,y1), color=draw_color, thickness=3)
            distance = math.dist((x0,y0), (x1,y1))
            if(distance<98):
                brush_thickness = 10
                eraser_thickness = 20
                thick_str = str(brush_thickness)
            elif(98<=distance<156):
                brush_thickness = 20
                eraser_thickness = 40
                thick_str = str(brush_thickness)
            elif(156<=distance<214):
                brush_thickness = 30
                eraser_thickness = 60
                thick_str = str(brush_thickness)
            elif(214<=distance<272):
                brush_thickness = 40
                eraser_thickness = 80
                thick_str = str(brush_thickness)
            elif(272<=distance):
                brush_thickness = 50
                eraser_thickness = 100
                thick_str = str(brush_thickness)
            if draw_color == (0,0,0):
                cv2.circle(frame, (x1,y1),int(eraser_thickness/2), draw_color, thickness=-1)
            else:
                cv2.circle(frame, (x1,y1),int(brush_thickness/2), draw_color, thickness=-1)
            
        if (my_fingers[1] and my_fingers[2] and not my_fingers[0] and not my_fingers[3] and not my_fingers[4]):
            xp = 0
            yp = 0
            max_min_count -= 1
            save_delay = 0
            fist = 0
            motion_count = []
            cv2.putText(frame, 'Selection Mode', (text_select_w,text_h), fontFace=cv2.FONT_HERSHEY_COMPLEX, color= (0,255,255), thickness=2, fontScale=1)
            cv2.line(frame, (x1,y1), (x2,y2), color=draw_color, thickness=3)

            xp, yp = 0,0

            if (y1<y1_read):
                if (max_min_l<x1<max_min_r and h_px == 720 and max_min_count <= 0): #maximize
                    max_min_count = 20
                    h_px = 1080
                    w_px = 1920
                    h_px_overlay_upto = 195
                    w_px_overlay_upto = 1920
                    y1_read = 205
                    max_min_l = 5
                    max_min_r = 173
                    cls_l = 190
                    cls_r = 326
                    sav_l = 346
                    sav_r = 465
                    thick_dec_l = 492
                    thick_dec_r = 550
                    thick_inc_l = 650
                    thick_inc_r = 708
                    white_l = 775
                    white_r = 879
                    yellow_l = 913
                    yellow_r = 1017
                    red_l = 1046
                    red_r = 1150
                    green_l = 1180
                    green_r = 1284
                    blue_l = 1316
                    blue_r = 1420
                    black_l = 1452
                    black_r = 1556
                    erase_l = 1620
                    erase_r = 1743
                    exit_l = 1782
                    exit_r = 1920
                    
                    text_write_w = 1682
                    text_select_w = 1653
                    text_client_w = 5
                    text_size_w = 1568
                    text_h = 1040
                    text_font_w = 576
                    text_font_h = 140
                    text_paint_save_h = 575
                    text_paint_save_w = 590
                    size_scale = 1

                    image_canvas =np.zeros((h_px,w_px,channel), np.uint8)
                    header_img = "Images1080"
                    header_img_list = os.listdir(header_img)
                    overlay_image =[]
                    for i in header_img_list:
                        image = cv2.imread(f'{header_img}/{i}')
                        overlay_image.append(image)
                    default_overlay= overlay_image[12]
                    cv2.namedWindow('paint', cv2.WND_PROP_FULLSCREEN)
                    # cv2.setWindowProperty('paint', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
                    frame = rescale_frame(frame, 150)

                elif (max_min_l<x1<max_min_r and h_px == 1080 and max_min_count <= 0): #minimize
                    max_min_count = 20
                    h_px = 720
                    w_px = 1280
                    h_px_overlay_upto = 130
                    w_px_overlay_upto = 1280
                    y1_read = 140
                    max_min_l = 5
                    max_min_r = 115
                    cls_l = 125
                    cls_r = 219
                    sav_l = 230
                    sav_r = 308
                    thick_dec_l = 326
                    thick_dec_r = 368
                    thick_inc_l = 432
                    thick_inc_r = 474
                    white_l = 508
                    white_r = 589
                    yellow_l = 603
                    yellow_r = 680
                    red_l = 694
                    red_r = 770
                    green_l = 784
                    green_r = 859
                    blue_l = 875
                    blue_r = 950
                    black_l = 967
                    black_r = 1039
                    erase_l = 1065
                    erase_r = 1165
                    exit_l = 1200
                    exit_r = 1280
                    
                    text_write_w = 1055
                    text_select_w = 1020
                    text_client_w = 5
                    text_size_w = 935
                    text_h = 712
                    text_font_w = 387
                    text_font_h = 91
                    text_paint_save_h = 430
                    text_paint_save_w = 255
                    size_scale = 0.5

                    ret, frame = cap.read()
                    frame = cv2.flip(frame,1)
                    image_canvas =np.zeros((h_px,w_px,channel), np.uint8)
                    header_img = "Images720"
                    header_img_list = os.listdir(header_img)
                    overlay_image =[]
                    for i in header_img_list:
                        image = cv2.imread(f'{header_img}/{i}')
                        overlay_image.append(image)
                    default_overlay= overlay_image[12]

                elif (white_l<x1<white_r): #white
                    default_overlay = overlay_image[3]
                    draw_color = (255,255,255)

                elif (yellow_l<x1<yellow_r): #yellow
                    default_overlay = overlay_image[4]
                    draw_color = (0,255,255,255)

                elif (red_l<x1<red_r): #red
                    default_overlay = overlay_image[5]
                    draw_color = (0,0,255)

                elif (green_l<x1<green_r): #green
                    default_overlay = overlay_image[6]
                    draw_color = (0,255,0)

                elif (blue_l<x1<blue_r): #blue
                    default_overlay = overlay_image[7]
                    draw_color = (255,80,40)

                elif (black_l<x1<black_r): #black
                    default_overlay = overlay_image[8]
                    draw_color = (51,51,51)

                elif (erase_l<x1<erase_r): #eraser
                    default_overlay = overlay_image[9]
                    draw_color = (0,0,0)

        if (my_fingers[1] and not my_fingers[0] and not my_fingers[2] and not my_fingers[3] and not my_fingers[4]):
            save_delay = 0
            fist = 0
            motion_count = []
            cv2.putText(frame, "Writing Mode", (text_write_w,text_h), fontFace= cv2.FONT_HERSHEY_COMPLEX, color= (255,255,0), thickness=2, fontScale=1)
            
            if xp ==0 and yp ==0:
                xp =x1 
                yp =y1

            if draw_color == (0,0,0):
                cv2.line(frame, (xp,yp),(x1,y1),color= draw_color, thickness=eraser_thickness)
                cv2.line(image_canvas, (xp,yp),(x1,y1),color= draw_color, thickness=eraser_thickness)
                cv2.circle(frame, (x1,y1),int(eraser_thickness/2), draw_color, thickness=-1)
                

            else:
                cv2.line(frame, (xp,yp),(x1,y1),color= draw_color, thickness=brush_thickness)
                cv2.line(image_canvas, (xp,yp),(x1,y1),color= draw_color, thickness=brush_thickness)
                cv2.circle(frame, (x1,y1),int(brush_thickness/2), draw_color, thickness=-1)
            xp , yp = x1, y1

        if(not my_fingers[0] and not my_fingers[1] and not my_fingers[2] and not my_fingers[3] and not my_fingers[4]):
            xp = 0
            yp = 0
            save_delay = 0
            motion_count = []
            fist+=1

        if(fist == 10):
            cap.release()
            cv2.destroyAllWindows()
            break

    img_gray = cv2.cvtColor(image_canvas, cv2.COLOR_BGR2GRAY)
    _, imginv= cv2.threshold(img_gray, 50, 255, cv2.THRESH_BINARY_INV)
    imginv = cv2.cvtColor(imginv, cv2.COLOR_GRAY2BGR)
    frame = cv2.bitwise_and(frame, imginv)
    frame =cv2.bitwise_or(frame, image_canvas)
    currentT = time.time()
    fps = 1/(currentT- previousT)
    previousT = currentT

    cv2.putText(frame, 'Client FPS:' + str(int(fps)), (text_client_w,text_h), fontFace=cv2.FONT_HERSHEY_SIMPLEX, color=(255,0,0), thickness=2, fontScale=1)

    cv2.imshow('paint',frame)
    cv2.waitKey(1)

['10.PNG', '11.PNG', '12.PNG', '13.PNG', '14.PNG', '15.PNG', '16.PNG', '17.PNG', '18.PNG', '19.PNG', '20.PNG', '21.PNG', '22.PNG']
